<font size=5>**Steps to build a Ranked Retrieval System**</font>
1. Load the Dataset
2. Text Pre-processing
3. Ranking Documents and Evaluation using MAP
  1. Jaccard Coefficient
  2. Term Frequency
  3. Inverse Document Frequency
  4. TF-IDF
  5. TF-IDF based Vector Space Model

In [1]:
# mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Importing Libraries
import numpy as np
import pandas as pd
import re
import spacy

# Loading model
nlp=spacy.load('en_core_web_sm')

# 1.Load the Dataset

## 1.1 Documents

In [3]:
# Loading documents
documents=pd.read_csv('/content/drive/My Drive/dataset/documents.csv')
print('Shape=>',documents.shape)
documents.head()

Shape=> (387, 5)


,docid,author,bibliography,body,title
0,2,ting-yili,"department of aeronautical engineering, rensse...",simple shear flow past a flat plate in an inco...,simple shear flow past a flat plate in an inco...
1,3,m. b. glauert,"department of mathematics, university of manch...",the boundary layer in simple shear flow past a...,the boundary layer in simple shear flow past a...
2,5,"wasserman,b.","j. ae. scs. 24, 1957, 924.",one-dimensional transient heat conduction into...,one-dimensional transient heat conduction into...
3,6,"campbell,w.f.","j. ae. scs. 25, 1958, 340.",one-dimensional transient heat flow in a multi...,one-dimensional transient heat flow in a multi...
4,12,"bisplinghoff,r.l.","j. ae. scs. 23, 1956, 289.",some structural and aerelastic considerations ...,some structural and aerelastic considerations ...


## 1.2 Queries

In [4]:
# Loading queries
queries=pd.read_csv('/content/drive/My Drive/dataset/queries.csv')
print('Shape=>',queries.shape)
queries.head()

Shape=> (85, 2)


,qid,query
0,1,what similarity laws must be obeyed when const...
1,2,what are the structural and aeroelastic proble...
2,3,what problems of heat conduction in composite ...
3,8,what methods -dash exact or approximate -dash ...
4,10,are real-gas transport properties for air avai...


## 1.3 Qrel

In [5]:
# Loading qrel
qrel=pd.read_csv('/content/drive/My Drive/dataset/qrel.csv')
print('Shape=>',qrel.shape)
qrel.head(10)

Shape=> (425, 2)


,qid,docid
0,1,184
1,1,29
2,1,31
3,1,57
4,1,378
5,2,12
6,2,746
7,2,15
8,2,184
9,2,858


## 1.4 Importing Validation Queries and Qrel

In [6]:
# Loading validation queries
queries_val=pd.read_csv('/content/drive/My Drive/dataset/queries_val.csv')
print('Shape=>',queries_val.shape)
queries_val.head()

Shape=> (22, 2)


,qid,query
0,189,is there a design method for calculating therm...
1,190,will an analysis of panel flutter based on arb...
2,191,"what is the criterion for true panel flutter, ..."
3,194,how can the analytical solution of the bucklin...
4,196,the problem of similarity for representative i...


In [7]:
# Loading qrel
qrel_val=pd.read_csv('/content/drive/My Drive/dataset/qrel_val.csv')
print('Shape=>',qrel_val.shape)
qrel_val.head(10)

Shape=> (110, 2)


,qid,docid
0,189,395
1,189,866
2,189,869
3,189,865
4,189,868
5,190,15
6,190,391
7,190,285
8,190,390
9,190,864


## 1.5 Reading samples

### 1.5.1 Reading sample queries

In [8]:
queries['query'].sample(10).values

array(['what are the structural and aeroelastic problems associated with flight of high speed aircraft .',
       'are there any theoretical methods for predicting base pressure .',
       'references on the methods available for accurately estimating aerodynamic heat transfer to conical bodies for both laminar and turbulent flow .',
       'does the boundary layer on a flat plate in a shear flow induce a pressure gradient .',
       'what qualitative and quantitative material is available on ablation materials research .',
       'jet interference with supersonic flow  -dash experimental papers .',
       'work on flow in channels at low reynolds numbers .',
       'what papers are there dealing with circumferential buckling either thermal buckling or due to mechanical loading .',
       'can series expansions be found for the boundary layer on a flat plate in a shear flow .',
       'have wind tunnel interference effects been investigated on a systematic basis .'],
      dtype=object

### 1.5.2 Reading sample documents

In [9]:
documents['body'][10:15].values

array(['a simple model study of transient temperature and thermal stress distribution due to aerodynamic heating .   the present work is concerned with the determination of transient temperatures and thermal stresses in simple models intended to simulate parts or the whole of an aircraft structure of the built- up variety subjected to aerodynamic heating .   the first case considered is that of convective heat transfer into one side of a flat plate, representing a thick skin, and the effect of the resulting temperature distribution in inducing thermal stresses associated with bending restraint at the plate edges . numerical results are presented for the transient temperature differentials in the plate when the environment temperature first increases linearly with time and then remains constant, the period of linear increase representing the time of acceleration of the aircraft .  corresponding thermal stress information is presented .   the second case is that of the wide-flanged i-bea

# 2.Text Pre-processing

In [10]:
def preprocess(text):
  # split on hyphen
  text=re.sub("-"," ",text)
  
  # keep only the words
  text=re.sub("[^a-z ]+","",text)
  
  # removing extra spaces
  text=re.sub("[\s]+"," ",text)

  # creating doc object
  doc=nlp(text)
  
  # remove stopwords and lemmatize the text
  tokens=[token.lemma_ for token in doc if(token.is_stop==False)]

  return tokens

In [11]:
# Pre-processing documents
documents['tokens']=documents['body'].apply(preprocess)
documents.head()

,docid,author,bibliography,body,title,tokens
0,2,ting-yili,"department of aeronautical engineering, rensse...",simple shear flow past a flat plate in an inco...,simple shear flow past a flat plate in an inco...,"[simple, shear, flow, past, flat, plate, incom..."
1,3,m. b. glauert,"department of mathematics, university of manch...",the boundary layer in simple shear flow past a...,the boundary layer in simple shear flow past a...,"[boundary, layer, simple, shear, flow, past, f..."
2,5,"wasserman,b.","j. ae. scs. 24, 1957, 924.",one-dimensional transient heat conduction into...,one-dimensional transient heat conduction into...,"[dimensional, transient, heat, conduction, dou..."
3,6,"campbell,w.f.","j. ae. scs. 25, 1958, 340.",one-dimensional transient heat flow in a multi...,one-dimensional transient heat flow in a multi...,"[dimensional, transient, heat, flow, multilaye..."
4,12,"bisplinghoff,r.l.","j. ae. scs. 23, 1956, 289.",some structural and aerelastic considerations ...,some structural and aerelastic considerations ...,"[structural, aerelastic, consideration, high, ..."


In [12]:
# Pre-processing queries
queries['tokens']=queries['query'].apply(preprocess)
queries.head()

,qid,query,tokens
0,1,what similarity laws must be obeyed when const...,"[similarity, law, obey, construct, aeroelastic..."
1,2,what are the structural and aeroelastic proble...,"[structural, aeroelastic, problem, associate, ..."
2,3,what problems of heat conduction in composite ...,"[problem, heat, conduction, composite, slab, s..."
3,8,what methods -dash exact or approximate -dash ...,"[method, dash, exact, approximate, dash, prese..."
4,10,are real-gas transport properties for air avai...,"[real, gas, transport, property, air, availabl..."


In [13]:
# Pre process validation queries
queries_val['tokens']=queries_val['query'].apply(preprocess)
queries_val.head()

,qid,query,tokens
0,189,is there a design method for calculating therm...,"[design, method, calculate, thermal, fatigue, ..."
1,190,will an analysis of panel flutter based on arb...,"[analysis, panel, flutter, base, arbitrarily, ..."
2,191,"what is the criterion for true panel flutter, ...","[criterion, true, panel, flutter, oppose, smal..."
3,194,how can the analytical solution of the bucklin...,"[analytical, solution, buckle, strength, unifo..."
4,196,the problem of similarity for representative i...,"[problem, similarity, representative, investig..."


# 3.Ranking Documents and Evaluation using MAP

## 3.1 Jaccard Coefficient
\\[\Large Jaccard(A,B)=\frac{\left | A\cap B \right |}{\left | A \cup B \right |} \\]

In [14]:
# Creating a temporary dataframe
temp_doc=documents[['docid','tokens']].copy()

In [15]:
def jaccard_coefficient(dtokens,qtokens):
  # Calculating |A intersection B|
  numerator=len(set(dtokens).intersection(set(qtokens)))
  # Calculating |A union B|
  denominator=len(set(dtokens).union(set(qtokens)))

  return numerator/denominator

In [16]:
# Calculating Jaccard Coefficient for a sample query-document pair
jaccard_coefficient(temp_doc['tokens'][0],queries['tokens'][0])

0.02702702702702703

In [17]:
# Getting jaccard coefficient for all the documents for against a sample query
temp_doc['jaccard']=temp_doc['tokens'].apply(lambda x: jaccard_coefficient(x,queries['tokens'][0]))
temp_doc.head(10)

,docid,tokens,jaccard
0,2,"[simple, shear, flow, past, flat, plate, incom...",0.027027
1,3,"[boundary, layer, simple, shear, flow, past, f...",0.000000
2,5,"[dimensional, transient, heat, conduction, dou...",0.000000
3,6,"[dimensional, transient, heat, flow, multilaye...",0.000000
4,12,"[structural, aerelastic, consideration, high, ...",0.066667
5,15,"[dimensional, panel, flutter, theory, experime...",0.000000
6,16,"[transformation, compressible, turbulent, boun...",0.000000
7,21,"[heat, transfer, slip, flow, number, author, c...",0.000000
8,23,"[skin, friction, heat, transfer, characteristi...",0.000000
9,24,"[theory, stagnation, point, heat, transfer, di...",0.021277


In [18]:
# DocIDs of the top 10 Most relevant documents
temp_doc.sort_values(by='jaccard',ascending=False).head(10).reset_index(drop=True)

,docid,tokens,jaccard
0,51,"[theory, aircraft, structural, model, subject,...",0.083333
1,875,"[model, aeroelastic, investigation, addendum, ...",0.066667
2,12,"[structural, aerelastic, consideration, high, ...",0.066667
3,184,"[scale, model, thermo, aeroelastic, research, ...",0.057971
4,1111,"[research, high, speed, flutter, paper, presen...",0.057143
5,1305,"[propose, programme, wind, tunnel, test, hyper...",0.055556
6,876,"[flutter, testing, high, speed, wind, tunnel, ...",0.051724
7,293,"[recent, study, effect, cool, boundary, layer,...",0.050000
8,67,"[dynamic, stability, vehicle, traverse, ascend...",0.048780
9,378,"[engineering, relation, friction, heat, transf...",0.047619


In [19]:
# DocIDs  of top 5 Most relevant documents
temp_doc.sort_values(by='jaccard',ascending=False).head()['docid'].values

array([  51,  875,   12,  184, 1111])

In [20]:
# Function for finding jaccard_coefficient
def jaccard_rank(qtokens):
  # Find jaccard coefficient for all docs
  temp_doc['jaccard']=temp_doc['tokens'].apply(lambda x: jaccard_coefficient(x,qtokens))

  # Find top 5 most relevant docs
  relevant_docids=temp_doc.sort_values(by='jaccard',ascending=False).head()['docid'].values

  return relevant_docids

In [21]:
# Ranking documents according to jaccard coefficient
queries['jaccard_rel']=queries['tokens'].apply(lambda x: jaccard_rank(x))
queries.head()

,qid,query,tokens,jaccard_rel
0,1,what similarity laws must be obeyed when const...,"[similarity, law, obey, construct, aeroelastic...","[51, 875, 12, 184, 1111]"
1,2,what are the structural and aeroelastic proble...,"[structural, aeroelastic, problem, associate, ...","[12, 51, 700, 746, 875]"
2,3,what problems of heat conduction in composite ...,"[problem, heat, conduction, composite, slab, s...","[5, 584, 6, 145, 582]"
3,8,what methods -dash exact or approximate -dash ...,"[method, dash, exact, approximate, dash, prese...","[122, 1306, 639, 655, 988]"
4,10,are real-gas transport properties for air avai...,"[real, gas, transport, property, air, availabl...","[405, 302, 436, 616, 583]"


### 3.1.1 Evaluation on Train Set


In [22]:
# Adding ground truth in a column
queries['ground_truth']=queries['qid'].apply(lambda x:qrel[qrel['qid']==x]['docid'].values)
queries.head()

,qid,query,tokens,jaccard_rel,ground_truth
0,1,what similarity laws must be obeyed when const...,"[similarity, law, obey, construct, aeroelastic...","[51, 875, 12, 184, 1111]","[184, 29, 31, 57, 378]"
1,2,what are the structural and aeroelastic proble...,"[structural, aeroelastic, problem, associate, ...","[12, 51, 700, 746, 875]","[12, 746, 15, 184, 858]"
2,3,what problems of heat conduction in composite ...,"[problem, heat, conduction, composite, slab, s...","[5, 584, 6, 145, 582]","[5, 6, 90, 91, 119]"
3,8,what methods -dash exact or approximate -dash ...,"[method, dash, exact, approximate, dash, prese...","[122, 1306, 639, 655, 988]","[48, 122, 354, 360, 1005]"
4,10,are real-gas transport properties for air avai...,"[real, gas, transport, property, air, availabl...","[405, 302, 436, 616, 583]","[259, 405, 302, 436, 437]"


\\[\Large Average\,Precision=\frac{\sum_{k=1}^{n}(P(k) \times rel(k))}{No.\,of\,relevant\,documents}\\]

In [23]:
def average_precision(model_rel,ground_truth):
  tp=0
  precisions=[]

  # Finding precision at positions at which relevant document is returned
  for index,value in enumerate(model_rel):
    if value in ground_truth:
      tp+=1
      precisions.append(tp/(index+1))

  # If no relevant document in list then return 0
  if precisions==[]:
    return 0
    
  return np.mean(precisions)

In [24]:
# Running on a sample
average_precision([5,6,1,2,4],[1,2,3,4,5])

0.8041666666666667

In [25]:

# Finding average precision for each query
queries['jaccard_ap']=queries.apply(lambda x: average_precision(x['jaccard_rel'],x['ground_truth']),axis=1)
queries.head()

,qid,query,tokens,jaccard_rel,ground_truth,jaccard_ap
0,1,what similarity laws must be obeyed when const...,"[similarity, law, obey, construct, aeroelastic...","[51, 875, 12, 184, 1111]","[184, 29, 31, 57, 378]",0.250000
1,2,what are the structural and aeroelastic proble...,"[structural, aeroelastic, problem, associate, ...","[12, 51, 700, 746, 875]","[12, 746, 15, 184, 858]",0.750000
2,3,what problems of heat conduction in composite ...,"[problem, heat, conduction, composite, slab, s...","[5, 584, 6, 145, 582]","[5, 6, 90, 91, 119]",0.833333
3,8,what methods -dash exact or approximate -dash ...,"[method, dash, exact, approximate, dash, prese...","[122, 1306, 639, 655, 988]","[48, 122, 354, 360, 1005]",1.000000
4,10,are real-gas transport properties for air avai...,"[real, gas, transport, property, air, availabl...","[405, 302, 436, 616, 583]","[259, 405, 302, 436, 437]",1.000000


In [26]:
# Finding Mean Average Precision
print('Mean Average Precision=>',queries['jaccard_ap'].mean())

Mean Average Precision=> 0.48960784313725514


### 3.1.2 Evaluation on Validation Set

In [27]:
# Adding ground truth in a column
queries_val['ground_truth']=queries_val['qid'].apply(lambda x:qrel_val[qrel_val['qid']==x]['docid'].values)

In [28]:
# Ranking documents according to jaccard coefficient
queries_val['jaccard_rel']=queries_val['tokens'].apply(lambda x: jaccard_rank(x))
queries_val.head()

,qid,query,tokens,ground_truth,jaccard_rel
0,189,is there a design method for calculating therm...,"[design, method, calculate, thermal, fatigue, ...","[395, 866, 869, 865, 868]","[868, 1306, 833, 906, 909]"
1,190,will an analysis of panel flutter based on arb...,"[analysis, panel, flutter, base, arbitrarily, ...","[15, 391, 285, 390, 864]","[390, 1008, 285, 21, 391]"
2,191,"what is the criterion for true panel flutter, ...","[criterion, true, panel, flutter, oppose, smal...","[914, 915, 285, 857, 858]","[285, 31, 864, 728, 914]"
3,194,how can the analytical solution of the bucklin...,"[analytical, solution, buckle, strength, unifo...","[739, 740, 742, 743, 744]","[932, 1050, 1172, 744, 1171]"
4,196,the problem of similarity for representative i...,"[problem, similarity, representative, investig...","[51, 185, 874, 875, 876]","[875, 1008, 184, 655, 180]"


In [29]:
# Finding average precision for each query
queries_val['jaccard_ap']=queries_val.apply(lambda x: average_precision(x['jaccard_rel'],x['ground_truth']),axis=1)
queries_val.head()

,qid,query,tokens,ground_truth,jaccard_rel,jaccard_ap
0,189,is there a design method for calculating therm...,"[design, method, calculate, thermal, fatigue, ...","[395, 866, 869, 865, 868]","[868, 1306, 833, 906, 909]",1.000000
1,190,will an analysis of panel flutter based on arb...,"[analysis, panel, flutter, base, arbitrarily, ...","[15, 391, 285, 390, 864]","[390, 1008, 285, 21, 391]",0.755556
2,191,"what is the criterion for true panel flutter, ...","[criterion, true, panel, flutter, oppose, smal...","[914, 915, 285, 857, 858]","[285, 31, 864, 728, 914]",0.700000
3,194,how can the analytical solution of the bucklin...,"[analytical, solution, buckle, strength, unifo...","[739, 740, 742, 743, 744]","[932, 1050, 1172, 744, 1171]",0.250000
4,196,the problem of similarity for representative i...,"[problem, similarity, representative, investig...","[51, 185, 874, 875, 876]","[875, 1008, 184, 655, 180]",1.000000


In [ ]:
# Finding Mean Average Precision
print('Mean Average Precision on Validation Set=>',queries_val['jaccard_ap'].mean())

Mean Average Precision on Validation Set=> 0.4232323232323232


## 3.2 Term Frequency
It is the count of each term present in a document and varies from document to document.

### 3.2.1 Computing Term Frequencies

In [30]:
vocabulary=set()

# Creating vocabulary
for i in documents['tokens'].values:
  vocabulary=vocabulary.union(set(i))

# sorting the vocabulary alphabetically
vocabulary=sorted(vocabulary)

In [31]:
print('Size of Vocabulary=>',len(vocabulary))

Size of Vocabulary=> 3068


In [32]:
tf_list_doc=[]

# Getting Term frequencies
for tokens in documents['tokens']:
  # Initliatizing a dictionary with 0 frequency
  doc_dict=dict.fromkeys(vocabulary,0)  
  
  # Counting term frequencies
  for term in tokens:
    doc_dict[term]+=1

  # Adding dictionary to list
  tf_list_doc.append(doc_dict)

In [33]:
len(tf_list_doc)

387

In [34]:
# Creating a dataframe of term frequencies for documents
documents_tf=pd.concat([documents['docid'],pd.DataFrame(tf_list_doc)],axis=1)
print('Shape=>',documents.shape)
documents_tf.head()

Shape=> (387, 6)


,docid,ab,abbreviate,ability,ablating,ablation,able,abrupt,abruptly,absence,absolute,absorb,absorption,academic,accelerate,accelerated,acceleration,accept,acceptability,acceptance,accessible,accidental,accommodate,accommodation,accompany,accompanying,accomplish,accord,accordance,accordingly,account,accumulation,accuracy,accurate,accurately,achieve,ackeret,acoustic,acoustical,acquisition,...,width,will,williams,wind,window,windward,wing,winglike,wire,withstand,wkb,womersley,wood,word,work,worker,working,worth,worthy,wrinkle,write,writer,x,xenon,xiii,y,yaw,yawed,yawing,year,yield,york,young,youngs,z,zbrozek,zero,zeroth,zone,zuk
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
# Getting Term Frequency for the first document in the dataset
print('Document ID:',documents_tf['docid'][0])
documents_tf.loc[0,documents_tf.loc[0,:]!=0][1:].to_dict()

Document ID: 2


{'approximation': 1,
 'arise': 1,
 'body': 2,
 'boundary': 5,
 'classical': 1,
 'consequently': 1,
 'consider': 2,
 'constant': 1,
 'curved': 1,
 'different': 1,
 'dimensional': 2,
 'discuss': 1,
 'discussion': 1,
 'edge': 1,
 'effect': 1,
 'emit': 1,
 'exist': 1,
 'feature': 1,
 'ferri': 1,
 'flat': 3,
 'flow': 6,
 'fluid': 2,
 'free': 3,
 'high': 1,
 'hypersonic': 2,
 'incompressible': 2,
 'instance': 1,
 'investigate': 1,
 'inviscid': 3,
 'irrotational': 1,
 'layer': 5,
 'lead': 1,
 'libby': 1,
 'necessary': 1,
 'nose': 1,
 'novel': 1,
 'original': 1,
 'outside': 1,
 'paper': 1,
 'past': 4,
 'plate': 3,
 'possible': 1,
 'prandtls': 2,
 'present': 1,
 'problem': 4,
 'recently': 1,
 'region': 1,
 'restrict': 1,
 'rotational': 2,
 'shear': 2,
 'shock': 2,
 'show': 1,
 'simple': 2,
 'situation': 2,
 'small': 2,
 'somewhat': 1,
 'speed': 1,
 'steady': 1,
 'stream': 3,
 'study': 2,
 'treat': 1,
 'usually': 1,
 'viscosity': 2,
 'viscous': 2,
 'vorticity': 2,
 'wave': 2}

<font size="5">Log Normalized Term Frequency,</font>
\\[\Large W_{t,d}=\begin{cases} 1+\log_{10}(tf_{t,d}) & ,\text{ if } tf>0 \\ 0 & ,\text{ Otherwise} \end{cases}\\]

In [36]:
# Function for Log Normalization
def log_normalize(x):
  if x!=0:
    return 1+np.log10(x)
  return 0

In [37]:
# Log Normalizing the term counts
documents_tf.iloc[:,1:]=documents_tf.iloc[:,1:].applymap(log_normalize)

In [38]:
# Getting Log normalized Term Frequency for the first document in the dataset
print('Document ID:',documents_tf['docid'][0])
documents_tf.loc[0,documents_tf.loc[0,:]!=0][1:].to_dict()

Document ID: 2


{'approximation': 1.0,
 'arise': 1.0,
 'body': 1.3010299956639813,
 'boundary': 1.6989700043360187,
 'classical': 1.0,
 'consequently': 1.0,
 'consider': 1.3010299956639813,
 'constant': 1.0,
 'curved': 1.0,
 'different': 1.0,
 'dimensional': 1.3010299956639813,
 'discuss': 1.0,
 'discussion': 1.0,
 'edge': 1.0,
 'effect': 1.0,
 'emit': 1.0,
 'exist': 1.0,
 'feature': 1.0,
 'ferri': 1.0,
 'flat': 1.4771212547196624,
 'flow': 1.7781512503836436,
 'fluid': 1.3010299956639813,
 'free': 1.4771212547196624,
 'high': 1.0,
 'hypersonic': 1.3010299956639813,
 'incompressible': 1.3010299956639813,
 'instance': 1.0,
 'investigate': 1.0,
 'inviscid': 1.4771212547196624,
 'irrotational': 1.0,
 'layer': 1.6989700043360187,
 'lead': 1.0,
 'libby': 1.0,
 'necessary': 1.0,
 'nose': 1.0,
 'novel': 1.0,
 'original': 1.0,
 'outside': 1.0,
 'paper': 1.0,
 'past': 1.6020599913279625,
 'plate': 1.4771212547196624,
 'possible': 1.0,
 'prandtls': 1.3010299956639813,
 'present': 1.0,
 'problem': 1.602059991327

### 3.2.2 Ranking

In [39]:
# Getting a list of unique query terms which are present in vocabulary
qtokens=list(set(queries['tokens'][0]).intersection(vocabulary))
print('Tokens in Query:',len(queries['tokens'][0]),queries['tokens'][0])
print('Tokens in Vocabulary:',len(qtokens),qtokens)

Tokens in Query: 10 ['similarity', 'law', 'obey', 'construct', 'aeroelastic', 'model', 'heated', 'high', 'speed', 'aircraft']
Tokens in Vocabulary: 9 ['construct', 'law', 'aircraft', 'speed', 'aeroelastic', 'similarity', 'heated', 'high', 'model']


In [40]:
print('Tokens not in Vocabulary:',set(queries['tokens'][0]).difference(set(qtokens)))

Tokens not in Vocabulary: {'obey'}


In [41]:
# Creating list of columns to retrieve
columns=['docid']
columns.extend(qtokens)
print(columns)

['docid', 'construct', 'law', 'aircraft', 'speed', 'aeroelastic', 'similarity', 'heated', 'high', 'model']


In [42]:
# Retireving term frequencies for query terms
temp_doc=documents_tf.loc[:,columns].copy()
temp_doc.head()

,docid,construct,law,aircraft,speed,aeroelastic,similarity,heated,high,model
0,2,0.0,0.0,0.00000,1.00000,0.00000,0.0,0.0,1.00000,0.0
1,3,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.0
2,5,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.0
3,6,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.0
4,12,0.0,0.0,1.30103,1.60206,1.30103,0.0,0.0,1.60206,0.0


In [43]:
# Adding all the frequencies
temp_doc['tf_sum']=temp_doc[qtokens].sum(axis=1)
temp_doc.head()

,docid,construct,law,aircraft,speed,aeroelastic,similarity,heated,high,model,tf_sum
0,2,0.0,0.0,0.00000,1.00000,0.00000,0.0,0.0,1.00000,0.0,2.00000
1,3,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000
2,5,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000
3,6,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000
4,12,0.0,0.0,1.30103,1.60206,1.30103,0.0,0.0,1.60206,0.0,5.80618


In [44]:
# Sorting dataframe according to sum of TF and getting relevant docs
temp_doc.sort_values(by='tf_sum',ascending=False).head()['docid'].values

array([ 51,  12, 184, 876, 252])

In [45]:
def tf_rank(qtokens):
  # Getting a list of unique query terms which are present in vocabulary
  qtokens=list(set(qtokens).intersection(vocabulary))
 
  # Creating list of columns to retrieve
  columns=['docid']
  columns.extend(qtokens)

  # Retireving term frequencies for query terms
  temp_doc=documents_tf.loc[:,columns].copy()

  # Adding all the frequencies
  temp_doc['tf_sum']=temp_doc[qtokens].sum(axis=1)

  # Sorting dataframe according to sum of TF and getting relevant docs
  rel_docs=temp_doc.sort_values(by='tf_sum',ascending=False).head()['docid'].values

  return rel_docs

In [46]:
# Ranking documents according to term frequency
queries['tf_rel']=queries['tokens'].apply(lambda x: tf_rank(x))
queries.head()

,qid,query,tokens,jaccard_rel,ground_truth,jaccard_ap,tf_rel
0,1,what similarity laws must be obeyed when const...,"[similarity, law, obey, construct, aeroelastic...","[51, 875, 12, 184, 1111]","[184, 29, 31, 57, 378]",0.250000,"[51, 12, 184, 876, 252]"
1,2,what are the structural and aeroelastic proble...,"[structural, aeroelastic, problem, associate, ...","[12, 51, 700, 746, 875]","[12, 746, 15, 184, 858]",0.750000,"[12, 172, 51, 746, 798]"
2,3,what problems of heat conduction in composite ...,"[problem, heat, conduction, composite, slab, s...","[5, 584, 6, 145, 582]","[5, 6, 90, 91, 119]",0.833333,"[5, 980, 584, 91, 395]"
3,8,what methods -dash exact or approximate -dash ...,"[method, dash, exact, approximate, dash, prese...","[122, 1306, 639, 655, 988]","[48, 122, 354, 360, 1005]",1.000000,"[122, 234, 1104, 924, 1307]"
4,10,are real-gas transport properties for air avai...,"[real, gas, transport, property, air, availabl...","[405, 302, 436, 616, 583]","[259, 405, 302, 436, 437]",1.000000,"[302, 185, 616, 1009, 1313]"


### 3.2.3 Evaluation on Train Set

In [47]:
# Finding average precision for each query
queries['tf_ap']=queries.apply(lambda x: average_precision(x['tf_rel'],x['ground_truth']),axis=1)
queries.head()

,qid,query,tokens,jaccard_rel,ground_truth,jaccard_ap,tf_rel,tf_ap
0,1,what similarity laws must be obeyed when const...,"[similarity, law, obey, construct, aeroelastic...","[51, 875, 12, 184, 1111]","[184, 29, 31, 57, 378]",0.250000,"[51, 12, 184, 876, 252]",0.333333
1,2,what are the structural and aeroelastic proble...,"[structural, aeroelastic, problem, associate, ...","[12, 51, 700, 746, 875]","[12, 746, 15, 184, 858]",0.750000,"[12, 172, 51, 746, 798]",0.750000
2,3,what problems of heat conduction in composite ...,"[problem, heat, conduction, composite, slab, s...","[5, 584, 6, 145, 582]","[5, 6, 90, 91, 119]",0.833333,"[5, 980, 584, 91, 395]",0.750000
3,8,what methods -dash exact or approximate -dash ...,"[method, dash, exact, approximate, dash, prese...","[122, 1306, 639, 655, 988]","[48, 122, 354, 360, 1005]",1.000000,"[122, 234, 1104, 924, 1307]",1.000000
4,10,are real-gas transport properties for air avai...,"[real, gas, transport, property, air, availabl...","[405, 302, 436, 616, 583]","[259, 405, 302, 436, 437]",1.000000,"[302, 185, 616, 1009, 1313]",1.000000


In [48]:
# Finding Mean Average Precision
print('Mean Average Precision=>',queries['tf_ap'].mean())

Mean Average Precision=> 0.5846405228758174


### 3.2.4 Evaluation on Validation Set

In [49]:
# Ranking documents according to term frequency
queries_val['tf_rel']=queries_val['tokens'].apply(lambda x: tf_rank(x))

# Finding average precision for each query
queries_val['tf_ap']=queries_val.apply(lambda x: average_precision(x['tf_rel'],x['ground_truth']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision on Validation Set=>',queries_val['tf_ap'].mean())

Mean Average Precision on Validation Set=> 0.43604797979797977


## 3.3 Inverse Document Frequency
<font size='3'>Inverse Document Frequency for a term in a corpus is given by,</font>
\\[\Large idf_{t}=\log_{10}(\frac{N}{df_{t}}) \\] 

### 3.3.1 Computing Inverse Document Frequencies(IDF)

In [50]:
print('No. of Documents=>',documents.shape[0])

No. of Documents=> 387


In [51]:
# Term Frequency of documents
documents_tf.head()

,docid,ab,abbreviate,ability,ablating,ablation,able,abrupt,abruptly,absence,absolute,absorb,absorption,academic,accelerate,accelerated,acceleration,accept,acceptability,acceptance,accessible,accidental,accommodate,accommodation,accompany,accompanying,accomplish,accord,accordance,accordingly,account,accumulation,accuracy,accurate,accurately,achieve,ackeret,acoustic,acoustical,acquisition,...,width,will,williams,wind,window,windward,wing,winglike,wire,withstand,wkb,womersley,wood,word,work,worker,working,worth,worthy,wrinkle,write,writer,x,xenon,xiii,y,yaw,yawed,yawing,year,yield,york,young,youngs,z,zbrozek,zero,zeroth,zone,zuk
0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
# Initializing a dictionary to stores IDF values
idf_dict=dict.fromkeys(vocabulary,0)

# Count of non-zero values per column(Document Frequency)
non_zero_count=np.count_nonzero(documents_tf.iloc[:,1:],axis=0)

# Assigning IDF values
for term,document_frequency in zip(list(vocabulary),non_zero_count):
  idf_dict[term]=np.log10(documents.shape[0]/(document_frequency))

In [53]:
idf_dict

{'ab': 2.28668096935493,
 'abbreviate': 2.5877109650189114,
 'ability': 2.5877109650189114,
 'ablating': 2.28668096935493,
 'ablation': 1.985650973690949,
 'able': 2.28668096935493,
 'abrupt': 2.5877109650189114,
 'abruptly': 2.28668096935493,
 'absence': 2.28668096935493,
 'absolute': 1.8887409606828927,
 'absorb': 2.5877109650189114,
 'absorption': 2.5877109650189114,
 'academic': 2.5877109650189114,
 'accelerate': 2.110589710299249,
 'accelerated': 2.5877109650189114,
 'acceleration': 2.110589710299249,
 'accept': 2.110589710299249,
 'acceptability': 2.5877109650189114,
 'acceptance': 2.5877109650189114,
 'accessible': 2.5877109650189114,
 'accidental': 2.5877109650189114,
 'accommodate': 2.5877109650189114,
 'accommodation': 2.5877109650189114,
 'accompany': 2.28668096935493,
 'accompanying': 2.5877109650189114,
 'accomplish': 2.28668096935493,
 'accord': 1.7426129250046545,
 'accordance': 2.5877109650189114,
 'accordingly': 1.985650973690949,
 'account': 1.189770956346874,
 'accum

### 3.3.2 Ranking

In [54]:
# Creating a temporary dataframe
temp_doc=documents[['docid','tokens']].copy()

In [55]:
# Function for getting sum of IDF values for a query-document pair
def idf_sum(dtokens,qtokens):
  # Getting common terms in query and document
  common_term=set(dtokens).intersection(set(qtokens))

  # Getting IDF values for common terms
  idf_list=[ value for key,value in idf_dict.items() if key in common_term]
  
  return sum(idf_list)

In [56]:
def idf_rank(qtokens):
  # Getting sum of IDF values for all the quer-document pairs
  temp_doc['idf_sum']=temp_doc['tokens'].apply(lambda x: idf_sum(x,qtokens))

  # Sorting dataframe according to sum of IDF and getting relevant docs
  rel_docs=temp_doc.sort_values(by='idf_sum',ascending=False).head()['docid'].values

  return rel_docs

In [57]:
# Ranking documents according to inverse document frequency
queries['idf_rel']=queries['tokens'].apply(lambda x: idf_rank(x))
queries.head()

,qid,query,tokens,jaccard_rel,ground_truth,jaccard_ap,tf_rel,tf_ap,idf_rel
0,1,what similarity laws must be obeyed when const...,"[similarity, law, obey, construct, aeroelastic...","[51, 875, 12, 184, 1111]","[184, 29, 31, 57, 378]",0.250000,"[51, 12, 184, 876, 252]",0.333333,"[51, 184, 12, 601, 62]"
1,2,what are the structural and aeroelastic proble...,"[structural, aeroelastic, problem, associate, ...","[12, 51, 700, 746, 875]","[12, 746, 15, 184, 858]",0.750000,"[12, 172, 51, 746, 798]",0.750000,"[12, 172, 51, 746, 364]"
2,3,what problems of heat conduction in composite ...,"[problem, heat, conduction, composite, slab, s...","[5, 584, 6, 145, 582]","[5, 6, 90, 91, 119]",0.833333,"[5, 980, 584, 91, 395]",0.750000,"[5, 91, 625, 584, 90]"
3,8,what methods -dash exact or approximate -dash ...,"[method, dash, exact, approximate, dash, prese...","[122, 1306, 639, 655, 988]","[48, 122, 354, 360, 1005]",1.000000,"[122, 234, 1104, 924, 1307]",1.000000,"[122, 556, 1104, 234, 924]"
4,10,are real-gas transport properties for air avai...,"[real, gas, transport, property, air, availabl...","[405, 302, 436, 616, 583]","[259, 405, 302, 436, 437]",1.000000,"[302, 185, 616, 1009, 1313]",1.000000,"[302, 332, 405, 1009, 583]"


### 3.3.3 Evaluation on Train Set

In [58]:
# Finding average precision for each query
queries['idf_ap']=queries.apply(lambda x: average_precision(x['idf_rel'],x['ground_truth']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision=>',queries['idf_ap'].mean())

Mean Average Precision=> 0.6256372549019611


### 3.3.4 Evaluation on Validation Set

In [59]:
# Ranking documents according to inverse document frequency
queries_val['idf_rel']=queries_val['tokens'].apply(lambda x: idf_rank(x))

# Finding average precision for each query
queries_val['idf_ap']=queries_val.apply(lambda x: average_precision(x['idf_rel'],x['ground_truth']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision on Validation Set=>',queries_val['idf_ap'].mean())

Mean Average Precision on Validation Set=> 0.353030303030303


## 3.4 TF-IDF
TF-IDF value for a term in a document is given by the product of Term Frequency and Inverse Document Frequency, i.e.,
\\[\Large W_{t,d}=[1+\log_{10}(tf_{t,d})]\times[\log_{10}(\frac{N}{df_{t}})]\\]


### 3.4.1 Computing TF-IDF

In [60]:
# Calculating TF-IDF
documents_tfidf=documents_tf.iloc[:,1:]*list(idf_dict.values())
documents_tfidf.head()

,ab,abbreviate,ability,ablating,ablation,able,abrupt,abruptly,absence,absolute,absorb,absorption,academic,accelerate,accelerated,acceleration,accept,acceptability,acceptance,accessible,accidental,accommodate,accommodation,accompany,accompanying,accomplish,accord,accordance,accordingly,account,accumulation,accuracy,accurate,accurately,achieve,ackeret,acoustic,acoustical,acquisition,acr,...,width,will,williams,wind,window,windward,wing,winglike,wire,withstand,wkb,womersley,wood,word,work,worker,working,worth,worthy,wrinkle,write,writer,x,xenon,xiii,y,yaw,yawed,yawing,year,yield,york,young,youngs,z,zbrozek,zero,zeroth,zone,zuk
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
# Adding a column of docids
documents_tfidf=pd.concat([documents['docid'],documents_tfidf],axis=1)
print('Shape=>',documents_tfidf.shape)
documents_tfidf.head()

Shape=> (387, 3069)


,docid,ab,abbreviate,ability,ablating,ablation,able,abrupt,abruptly,absence,absolute,absorb,absorption,academic,accelerate,accelerated,acceleration,accept,acceptability,acceptance,accessible,accidental,accommodate,accommodation,accompany,accompanying,accomplish,accord,accordance,accordingly,account,accumulation,accuracy,accurate,accurately,achieve,ackeret,acoustic,acoustical,acquisition,...,width,will,williams,wind,window,windward,wing,winglike,wire,withstand,wkb,womersley,wood,word,work,worker,working,worth,worthy,wrinkle,write,writer,x,xenon,xiii,y,yaw,yawed,yawing,year,yield,york,young,youngs,z,zbrozek,zero,zeroth,zone,zuk
0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
# Getting TF_IDF values for the terms in first document
print('Document ID:',documents_tfidf['docid'][0])
documents_tfidf.loc[0,documents_tfidf.loc[0,:]!=0][1:].to_dict()

Document ID: 2


{'approximation': 0.9249531333373373,
 'arise': 1.4415829293406734,
 'body': 0.8179179714411188,
 'boundary': 0.7825560609575195,
 'classical': 1.5463182798606863,
 'consequently': 2.110589710299249,
 'consider': 1.0167952618775644,
 'constant': 0.8887409606828927,
 'curved': 1.5463182798606863,
 'different': 1.1405529336766922,
 'dimensional': 1.0532614720216564,
 'discuss': 0.7243881048984555,
 'discussion': 1.2074997233073055,
 'edge': 0.7488618742816561,
 'effect': 0.43237492755384954,
 'emit': 2.5877109650189114,
 'exist': 1.0314084642516241,
 'feature': 1.3835909823629866,
 'ferri': 2.28668096935493,
 'flat': 1.1061597913506378,
 'flow': 0.5293135804637596,
 'fluid': 1.3741895767992272,
 'free': 1.1444700154382332,
 'high': 0.5964848893264165,
 'hypersonic': 1.1024255232317401,
 'incompressible': 1.311343739741794,
 'instance': 1.8887409606828927,
 'investigate': 1.0195092409519164,
 'inviscid': 1.8692846438486488,
 'irrotational': 1.985650973690949,
 'layer': 0.8049169630236986,

### 3.4.2 Ranking

In [63]:
def tf_idf_rank(qtokens):
  # Getting a list of unique query terms which are present in vocabulary
  qtokens=list(set(qtokens).intersection(vocabulary))

  # Creating list of columns to retrieve
  columns=['docid']
  columns.extend(qtokens)

  # Retireving TF-IDF for query terms
  temp_doc=documents_tfidf.loc[:,columns].copy()

  # Adding all the frequencies
  temp_doc['tfidf_sum']=temp_doc[qtokens].sum(axis=1)

  # Sorting dataframe according to sum of TF-IDF and getting relevant docs
  rel_docs=temp_doc.sort_values(by='tfidf_sum',ascending=False).head()['docid'].values

  return rel_docs

In [64]:
# Ranking documents according to TF-IDF
queries['tfidf_rel']=queries['tokens'].apply(lambda x: tf_idf_rank(x))
queries.head()

,qid,query,tokens,jaccard_rel,ground_truth,jaccard_ap,tf_rel,tf_ap,idf_rel,idf_ap,tfidf_rel
0,1,what similarity laws must be obeyed when const...,"[similarity, law, obey, construct, aeroelastic...","[51, 875, 12, 184, 1111]","[184, 29, 31, 57, 378]",0.250000,"[51, 12, 184, 876, 252]",0.333333,"[51, 184, 12, 601, 62]",0.500000,"[51, 184, 12, 332, 746]"
1,2,what are the structural and aeroelastic proble...,"[structural, aeroelastic, problem, associate, ...","[12, 51, 700, 746, 875]","[12, 746, 15, 184, 858]",0.750000,"[12, 172, 51, 746, 798]",0.750000,"[12, 172, 51, 746, 364]",0.750000,"[12, 51, 172, 746, 724]"
2,3,what problems of heat conduction in composite ...,"[problem, heat, conduction, composite, slab, s...","[5, 584, 6, 145, 582]","[5, 6, 90, 91, 119]",0.833333,"[5, 980, 584, 91, 395]",0.750000,"[5, 91, 625, 584, 90]",0.866667,"[5, 91, 90, 584, 625]"
3,8,what methods -dash exact or approximate -dash ...,"[method, dash, exact, approximate, dash, prese...","[122, 1306, 639, 655, 988]","[48, 122, 354, 360, 1005]",1.000000,"[122, 234, 1104, 924, 1307]",1.000000,"[122, 556, 1104, 234, 924]",1.000000,"[122, 234, 1104, 556, 1307]"
4,10,are real-gas transport properties for air avai...,"[real, gas, transport, property, air, availabl...","[405, 302, 436, 616, 583]","[259, 405, 302, 436, 437]",1.000000,"[302, 185, 616, 1009, 1313]",1.000000,"[302, 332, 405, 1009, 583]",0.833333,"[302, 1009, 185, 583, 332]"


### 3.4.3 Evaluation on Train Set

In [65]:
# Finding average precision for each query
queries['tfidf_ap']=queries.apply(lambda x: average_precision(x['tfidf_rel'],x['ground_truth']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision=>',queries['tfidf_ap'].mean())

Mean Average Precision=> 0.6612745098039219


### 3.4.4 Evaluation on Validation Set

In [66]:
# Ranking documents according to TF-IDF
queries_val['tfidf_rel']=queries_val['tokens'].apply(lambda x: tf_idf_rank(x))

# Finding average precision for each query
queries_val['tfidf_ap']=queries_val.apply(lambda x: average_precision(x['tfidf_rel'],x['ground_truth']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision on Validation Set=>',queries_val['tfidf_ap'].mean())

Mean Average Precision on Validation Set=> 0.42051767676767665


## 3.5 TF-IDF based Vector Space Model

### 3.5.1 Creating TF-IDF vectors for Queries

In [67]:
def gen_tfidf_queries(queries_data):
  tf_list_queries=[]

  # Getting Term frequencies
  for tokens in queries_data['tokens']:
    # Initliatizing a dictionary with 0 frequency
    queries_dict=dict.fromkeys(vocabulary,0)      
    # Counting term frequencies
    for term in set(tokens).intersection(vocabulary):
      queries_dict[term]+=1
    # Adding dictionary to list
    tf_list_queries.append(queries_dict)

  # Creating a dataframe of term frequencies for queries
  queries_tf=pd.DataFrame(tf_list_queries)

  # Log Normalizing the term counts for queries
  queries_tf=queries_tf.applymap(log_normalize)

  # Calculating TF-IDF for queries
  queries_tfidf=queries_tf*list(idf_dict.values())

  # Adding a column of qids
  queries_tfidf=pd.concat([queries_data['qid'],queries_tfidf],axis=1)

  return queries_tfidf

In [68]:
# Creating TF-IDF vectors for queries in train set
queries_tfidf=gen_tfidf_queries(queries)
print('Shape=>',queries_tfidf.shape)
queries_tfidf.head()

Shape=> (85, 3069)


,qid,ab,abbreviate,ability,ablating,ablation,able,abrupt,abruptly,absence,absolute,absorb,absorption,academic,accelerate,accelerated,acceleration,accept,acceptability,acceptance,accessible,accidental,accommodate,accommodation,accompany,accompanying,accomplish,accord,accordance,accordingly,account,accumulation,accuracy,accurate,accurately,achieve,ackeret,acoustic,acoustical,acquisition,...,width,will,williams,wind,window,windward,wing,winglike,wire,withstand,wkb,womersley,wood,word,work,worker,working,worth,worthy,wrinkle,write,writer,x,xenon,xiii,y,yaw,yawed,yawing,year,yield,york,young,youngs,z,zbrozek,zero,zeroth,zone,zuk
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
# Creating TF-IDF vectors for queries in validation set
queries_val_tfidf=gen_tfidf_queries(queries_val)
print('Shape=>',queries_val_tfidf.shape)
queries_val_tfidf.head()

Shape=> (22, 3069)


,qid,ab,abbreviate,ability,ablating,ablation,able,abrupt,abruptly,absence,absolute,absorb,absorption,academic,accelerate,accelerated,acceleration,accept,acceptability,acceptance,accessible,accidental,accommodate,accommodation,accompany,accompanying,accomplish,accord,accordance,accordingly,account,accumulation,accuracy,accurate,accurately,achieve,ackeret,acoustic,acoustical,acquisition,...,width,will,williams,wind,window,windward,wing,winglike,wire,withstand,wkb,womersley,wood,word,work,worker,working,worth,worthy,wrinkle,write,writer,x,xenon,xiii,y,yaw,yawed,yawing,year,yield,york,young,youngs,z,zbrozek,zero,zeroth,zone,zuk
0,189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,190,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,191,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.888741,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.286681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 3.5.2 Ranking

In [70]:
# Temporary dataframe with tfidf values
temp_doc_tfidf=documents_tfidf.copy()

Cosine Similarity for two vectors A and B is given by,
\\[\Large Cosine \, Simiarity (A,B)=\frac{\overrightarrow{A}\cdot\overrightarrow{B}}{\left \| \overrightarrow{A} \right \|\left \| \overrightarrow{B} \right \|}\\]

In [71]:
from sklearn.metrics.pairwise import cosine_similarity

In [72]:
queries_tfidf.iloc[0,1:].values

array([0., 0., 0., ..., 0., 0., 0.])

In [73]:
temp_doc_tfidf.iloc[0,1:].values

array([0., 0., 0., ..., 0., 0., 0.])

In [74]:
cosine_similarity(queries_tfidf.iloc[0,1:].values.reshape(1,-1),temp_doc_tfidf.iloc[0,1:].values.reshape(1,-1))

array([[0.01458162]])

In [75]:
cosine_similarity(queries_tfidf.iloc[0,1:].values.reshape(1,-1),temp_doc_tfidf.iloc[0,1:].values.reshape(1,-1)).item()

0.014581616635538785

In [76]:
def tfidf_vsm_rank(queries_data):
  # Finding cosine similarity score for every document vector against a query vector
  temp_doc_tfidf['tfidf_vsm']=temp_doc_tfidf.apply(lambda x: cosine_similarity(x.values[1:].reshape(1, -1),queries_data[1:].values.reshape(1, -1)).item(),axis=1)
  
  # Sorting dataframe according to sum of cosine similarity score and getting relevant docs
  rel_docs=temp_doc_tfidf.sort_values(by='tfidf_vsm',ascending=False).head()['docid'].values

  # Droppping similarity column
  temp_doc_tfidf.drop(columns='tfidf_vsm',inplace=True)

  return rel_docs

In [77]:
# Ranking documents according to Cosine Similarity
queries['tfidf_vsm_rel']=queries_tfidf.apply(lambda x: tfidf_vsm_rank(x),axis=1)
queries.head()

,qid,query,tokens,jaccard_rel,ground_truth,jaccard_ap,tf_rel,tf_ap,idf_rel,idf_ap,tfidf_rel,tfidf_ap,tfidf_vsm_rel
0,1,what similarity laws must be obeyed when const...,"[similarity, law, obey, construct, aeroelastic...","[51, 875, 12, 184, 1111]","[184, 29, 31, 57, 378]",0.250000,"[51, 12, 184, 876, 252]",0.333333,"[51, 184, 12, 601, 62]",0.500000,"[51, 184, 12, 332, 746]",0.50,"[51, 184, 12, 875, 332]"
1,2,what are the structural and aeroelastic proble...,"[structural, aeroelastic, problem, associate, ...","[12, 51, 700, 746, 875]","[12, 746, 15, 184, 858]",0.750000,"[12, 172, 51, 746, 798]",0.750000,"[12, 172, 51, 746, 364]",0.750000,"[12, 51, 172, 746, 724]",0.75,"[12, 51, 875, 746, 184]"
2,3,what problems of heat conduction in composite ...,"[problem, heat, conduction, composite, slab, s...","[5, 584, 6, 145, 582]","[5, 6, 90, 91, 119]",0.833333,"[5, 980, 584, 91, 395]",0.750000,"[5, 91, 625, 584, 90]",0.866667,"[5, 91, 90, 584, 625]",1.00,"[5, 90, 91, 582, 584]"
3,8,what methods -dash exact or approximate -dash ...,"[method, dash, exact, approximate, dash, prese...","[122, 1306, 639, 655, 988]","[48, 122, 354, 360, 1005]",1.000000,"[122, 234, 1104, 924, 1307]",1.000000,"[122, 556, 1104, 234, 924]",1.000000,"[122, 234, 1104, 556, 1307]",1.00,"[556, 122, 569, 48, 608]"
4,10,are real-gas transport properties for air avai...,"[real, gas, transport, property, air, availabl...","[405, 302, 436, 616, 583]","[259, 405, 302, 436, 437]",1.000000,"[302, 185, 616, 1009, 1313]",1.000000,"[302, 332, 405, 1009, 583]",0.833333,"[302, 1009, 185, 583, 332]",1.00,"[302, 405, 436, 583, 1009]"


### 3.5.3 Evaluation on Train Set

In [78]:
# Finding average precision for each query
queries['tfidf_vsm_ap']=queries.apply(lambda x: average_precision(x['tfidf_vsm_rel'],x['ground_truth']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision=>',queries['tfidf_vsm_ap'].mean())

Mean Average Precision=> 0.6408823529411768


### 3.5.4 Evaluation on Validation Set

In [79]:
# Ranking documents according to Cosine Similarity
queries_val['tfidf_vsm_rel']=queries_val_tfidf.apply(lambda x: tfidf_vsm_rank(x),axis=1)

# Finding average precision for each query
queries_val['tfidf_vsm_ap']=queries_val.apply(lambda x: average_precision(x['tfidf_vsm_rel'],x['ground_truth']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision=>',queries_val['tfidf_vsm_ap'].mean())

Mean Average Precision=> 0.3277777777777777


# Summary

|   Method   | Train Set | Evaluation Set |
|:----------:|:---------:|:--------------:|
|   Jaccard  |   0.4896  |     0.4232     |
|     TF     |   0.5846  |     0.4360     |
|     IDF    |   0.6256  |     0.3530     |
|   TF-IDF   |   0.6613  |     0.4205     |
| TF-IDF VSM |   0.6409  |     0.3278     |